In [2]:
import numpy as np
import os
from matplotlib import pyplot as plt
import time

In [9]:
# Data Preprocessing & Label and Feature creation

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

DATA_PATH = os.path.join('MP_Data') 

actions = np.array(['hello', 'thanks', 'iloveyou'])

no_sequences = 30

sequence_length = 30

label_map = {label:num for num, label in enumerate(actions)}

sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [16]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.05)

In [20]:
# Build and Train LSTM Neural Network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

model.summary()

Metal device set to: Apple M2


2022-09-15 21:35:53.703531: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-15 21:35:53.704703: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/2000


2022-09-15 21:35:54.687765: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-15 21:35:55.896426: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 11s 2s/step - loss: 3.5605 - categorical_accuracy: 0.2588
Epoch 2/2000
3/3 [==============================] - 6s 2s/step - loss: 3.5681 - categorical_accuracy: 0.2588
Epoch 3/2000
3/3 [==============================] - 7s 2s/step - loss: 3.8320 - categorical_accuracy: 0.4118
Epoch 4/2000
3/3 [==============================] - 6s 2s/step - loss: 2.0162 - categorical_accuracy: 0.3882
Epoch 5/2000
3/3 [==============================] - 6s 2s/step - loss: 1.6627 - categorical_accuracy: 0.3412
Epoch 6/2000
3/3 [==============================] - 6s 2s/step - loss: 1.7236 - categorical_accuracy: 0.2471
Epoch 7/2000
3/3 [==============================] - 6s 2s/step - loss: 1.0720 - categorical_accuracy: 0.4471
Epoch 8/2000
3/3 [==============================] - 6s 2s/step - loss: 1.1273 - categorical_accuracy: 0.4824
Epoch 9/2000
3/3 [==============================] - 6s 2s/step - loss: 1.0958 - categorical_accuracy: 0.4235
Epoch 10/2000
3/3 [============

KeyboardInterrupt: 

In [23]:
# predict
res = model.predict(X_test)

1/1 [==============================] - 0s 164ms/step


In [24]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(X_test)
